In [1]:
from transformers import pipeline
from PIL import Image
import numpy as np

In [2]:
segmenter = pipeline("image-segmentation", model="mattmdjaga/segformer_b2_clothes")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# Segment Clothes

In [5]:
# Initialize segmentation pipeline
def segment_clothing(img, clothes= ["Upper-clothes", "Pants", "Belt"]):
    # Segment image
    segments = segmenter(img)

    # Create list of masks
    mask_list = []
    for s in segments:
        if(s['label'] in clothes):
            mask_list.append(s['mask'])


    # Paste all masks on top of eachother 
    final_mask = np.array(mask_list[0])
    for mask in mask_list:
        current_mask = np.array(mask)
        final_mask = final_mask + current_mask
            
    # Convert final mask from np array to PIL image
    final_mask = Image.fromarray(final_mask)

    # Apply mask to original image
    img.putalpha(final_mask)

    # display the image
    return img

In [6]:
def batch_segment_clothing(img_dir, out_dir, clothes= ["Hat", "Upper-clothes", "Skirt", "Pants", "Dress", "Belt", "Left-shoe", "Right-shoe", "Scarf"]):
    # Create output directory if it doesn't exist
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Iterate through each file in the input directory
    for filename in os.listdir(img_dir):
        if filename.endswith(".jpg") or filename.endswith(".JPG") or filename.endswith(".png") or filename.endswith(".PNG"):
            try:
                # Load image
                img_path = os.path.join(img_dir, filename)
                img = Image.open(img_path).convert("RGBA")

                # Segment clothing
                segmented_img = segment_clothing(img, clothes)

                # Save segmented image to output directory as PNG
                out_path = os.path.join(out_dir, filename.split('.')[0] + ".png")
                segmented_img.save(out_path)

                print(f"Segmented {filename} successfully.")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

        else:
            print(f"Skipping {filename} as it is not a supported image file.")

In [3]:
import urllib.request
from urllib.parse import unquote
import re

# URL of the file to download
url = "https://dtcralphlauren.scene7.com/is/image/PoloGSI/s7-1485490_lifestyle?$plpDeskRF$"

# Extract the filename from the URL
filename = url.split("/")[-1]  # Default fallback if Content-Disposition is not available
filename = unquote(filename)  # Decode URL-encoded characters in the filename

# Download and save the file
urllib.request.urlretrieve(url, filename)

print(f"File downloaded and saved as: {filename}")

# open the file
image=Image.open(filename)

File downloaded and saved as: s7-1485490_lifestyle?$plpDeskRF$


In [7]:
result = segment_clothing(img=image)
result.save(f'{filename}.png')

NameError: name 'image' is not defined

# read the colors